In [ ]:
import pygame
import sys
import numpy as np
from qiskit import QuantumCircuit, ClassicalRegister
from qiskit_aer import AerSimulator


# --- Quantum Circuit Logic ---
def apply_circuit(qc):
    # Make a copy of the circuit
    try:
        qc_copy = qc.copy()
    except AttributeError:
        # For older Qiskit versions
        qc_copy = QuantumCircuit.from_qasm_str(qc.qasm())
    # Check if measurement already exists
    has_measure = any(instr[0].name == "measure" for instr in qc_copy.data)
    if not has_measure:
        qc_copy.measure(0, 0)
        qc_copy.measure(1, 1)
    sim = AerSimulator()
    result = sim.run(qc_copy).result()
    counts = result.get_counts()
    # Return the most probable result
    measured = max(counts, key=counts.get)
    return measured
# --- Player Class ---

class Player:
    def __init__(self, name):
        self.name = name
        self.gates = {"H":1, "Z":1, "Y":1, "X":1, "CNOT":1, "SWAP":1}
    def get_gate(self):
        return dict(self.gates)

# --- Game Setup ---

pygame.init()
WIDTH, HEIGHT = 900, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Quantum Gate Minigame")

font = pygame.font.SysFont(None, 32)

players = [Player("Mario"), Player("Luigi"), Player("Yoshi"), Player("Peach")]
current_player = 0
skipped_players = set()

# --- Gate UI Setup ---

GATE_COLORS = {
    "H": (200,200,255),
    "Z": (255,200,200),
    "Y": (200,255,200),
    "X": (255,255,200),
    "CNOT": (200,255,255),
    "SWAP": (255,200,255)
}
GATE_LIST = ["H","Z","Y","X","CNOT","SWAP"]

gate_rects = {}
for i, gate in enumerate(GATE_LIST):
    gate_rects[gate] = pygame.Rect(30, 50 + i*60, 80, 40)

# --- Circuit State ---
# Use a Qiskit QuantumCircuit to store the circuit directly
MAX_GATES = 10
def create_empty_circuit():
    qc = QuantumCircuit(2, 2)
    # Start with both qubits in |0>
    # Apply H to both as in original logic
    qc.h(0)
    qc.h(1)
    return qc

qiskit_circuit = create_empty_circuit()
gate_history = [("H", 0), ("H", 1)]  # Show initial H on both qubits

def draw_gates(player):
    for i, gate in enumerate(GATE_LIST):
        rect = gate_rects[gate]
        # Highlight in green if current player
        if players[current_player] == player:
            pygame.draw.rect(screen, (0, 255, 0), rect, 4)
        pygame.draw.rect(screen, GATE_COLORS[gate], rect)
        txt = font.render(f"{gate} ({player.gates[gate]})", True, (0,0,0))
        screen.blit(txt, (rect.x+5, rect.y+5))

def draw_circuit():
    base_x = 200
    base_y = 150
    place_x = base_x + (len(gate_history)+1)*60
    for q in range(2):
        y = base_y + q*60
        # Always use ASCII |0> for maximum compatibility
        ket0_txt = font.render("|0>", True, (0,0,0))
        screen.blit(ket0_txt, (base_x-50, y-12))
        pygame.draw.line(screen, (0,0,0), (base_x, y), (base_x+MAX_GATES*60, y), 2)
        pygame.draw.rect(screen, (0,255,0), (place_x-20, y-20, 40, 40), 3)
    pygame.draw.rect(screen, (0,255,0), (place_x-20, base_y-20, 40, 100), 3)
    for idx, gate_info in enumerate(gate_history):
        x = base_x + (idx+1)*60
        if gate_info[0] == "CNOT":
            _, control, target = gate_info
            y1 = base_y + control*60
            y2 = base_y + target*60
            pygame.draw.circle(screen, (0,0,0), (x, y1), 8)
            pygame.draw.line(screen, (0,0,0), (x, y1), (x, y2), 2)
            pygame.draw.circle(screen, (0,0,0), (x, y2), 12, 2)
            pygame.draw.line(screen, (0,0,0), (x-10, y2), (x+10, y2), 2)
            pygame.draw.line(screen, (0,0,0), (x, y2-10), (x, y2+10), 2)
        elif gate_info[0] == "SWAP":
            _, q1, q2 = gate_info
            y1 = base_y + q1*60
            y2 = base_y + q2*60
            pygame.draw.line(screen, (0,0,0), (x-10, y1-10), (x+10, y1+10), 2)
            pygame.draw.line(screen, (0,0,0), (x-10, y1+10), (x+10, y1-10), 2)
            pygame.draw.line(screen, (0,0,0), (x-10, y2-10), (x+10, y2+10), 2)
            pygame.draw.line(screen, (0,0,0), (x-10, y2+10), (x+10, y2-10), 2)
            pygame.draw.line(screen, (0,0,0), (x, y1), (x, y2), 2)
        else:
            gate, qubit = gate_info
            y = base_y + qubit*60
            pygame.draw.rect(screen, GATE_COLORS.get(gate, (220,220,220)), (x-20, y-20, 40, 40))
            txt = font.render(gate, True, (0,0,0))
            screen.blit(txt, (x-10, y-10))

def draw_measure_button():
    btn_rect = pygame.Rect(WIDTH-180, HEIGHT-80, 150, 50)
    pygame.draw.rect(screen, (100,200,100), btn_rect)
    txt = font.render("Measurement", True, (0,0,0))
    screen.blit(txt, (btn_rect.x+10, btn_rect.y+10))
    return btn_rect

def draw_skip_button():
    btn_rect = pygame.Rect(WIDTH-350, HEIGHT-80, 120, 50)
    pygame.draw.rect(screen, (200,100,100), btn_rect)
    txt = font.render("Skip", True, (0,0,0))
    screen.blit(txt, (btn_rect.x+30, btn_rect.y+10))
    return btn_rect

def draw_player_info():
    txt = font.render(f"Current Player: {players[current_player].name}", True, (0,0,0))
    screen.blit(txt, (30, 10))

# --- Drag and Drop ---
dragging_gate = None
drag_offset = (0,0)
drag_pos = (0,0)

# --- Main Loop ---

measurement_result = None

def next_player():
    global current_player
    n = len(players)
    for _ in range(n):
        current_player = (current_player + 1) % n
        if current_player not in skipped_players:
            return

def reset_circuit():
    global qiskit_circuit, gate_history
    qiskit_circuit = create_empty_circuit()
    gate_history = [("H", 0), ("H", 1)]

while True:
    screen.fill((240,240,240))
    draw_player_info()
    draw_gates(players[current_player])
    draw_circuit()
    btn_rect = draw_measure_button()
    skip_btn_rect = draw_skip_button()
    if measurement_result:
        txt = font.render(f"Measured: {measurement_result}", True, (0,0,0))
        screen.blit(txt, (WIDTH-300, HEIGHT-140))
    # Draw dragging gate if any
    if dragging_gate:
        mx, my = drag_pos
        pygame.draw.rect(screen, GATE_COLORS[dragging_gate], (mx-40, my-20, 80, 40))
        txt = font.render(dragging_gate, True, (0,0,0))
        screen.blit(txt, (mx-20, my-10))
    pygame.display.flip()

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        elif event.type == pygame.MOUSEBUTTONDOWN:
            mx, my = event.pos
            if skip_btn_rect.collidepoint(mx, my):
                skipped_players.add(current_player)
                next_player()
                dragging_gate = None
                drag_pos = (0,0)
                continue
            for gate, rect in gate_rects.items():
                if rect.collidepoint(mx, my) and players[current_player].gates[gate] > 0:
                    dragging_gate = gate
                    drag_offset = (mx - rect.x, my - rect.y)
                    drag_pos = (mx, my)
            if btn_rect.collidepoint(mx, my):
                measurement_result = apply_circuit(qiskit_circuit)
        elif event.type == pygame.MOUSEBUTTONUP:
            if dragging_gate:
                mx, my = event.pos
                base_x = 200 + (len(gate_history)+1)*60
                dropped = False
                if dragging_gate == "CNOT":
                    for q in range(2):
                        y = 150 + q*60
                        if base_x-20 < mx < base_x+20 and y-20 < my < y+20:
                            control = q
                            target = 1 - q
                            qiskit_circuit.cx(control, target)
                            gate_history.append(("CNOT", control, target))
                            players[current_player].gates["CNOT"] -= 1
                            next_player()
                            measurement_result = None
                            dropped = True
                            break
                    dragging_gate = None
                    drag_pos = (0,0)
                elif dragging_gate == "SWAP":
                    qiskit_circuit.swap(0, 1)
                    gate_history.append(("SWAP", 0, 1))
                    players[current_player].gates["SWAP"] -= 1
                    next_player()
                    measurement_result = None
                    dragging_gate = None
                    drag_pos = (0,0)
                else:
                    for q in range(2):
                        y = 150 + q*60
                        if base_x-20 < mx < base_x+20 and y-20 < my < y+20:
                            if dragging_gate == "H":
                                qiskit_circuit.h(q)
                            elif dragging_gate == "X":
                                qiskit_circuit.x(q)
                            elif dragging_gate == "Y":
                                qiskit_circuit.y(q)
                            elif dragging_gate == "Z":
                                qiskit_circuit.z(q)
                            gate_history.append((dragging_gate, q))
                            players[current_player].gates[dragging_gate] -= 1
                            next_player()
                            measurement_result = None
                            dropped = True
                            break
                    dragging_gate = None
                    drag_pos = (0,0)
        elif event.type == pygame.MOUSEMOTION:
            if dragging_gate:
                drag_pos = event.pos


SystemExit: 